In [1]:
class treeNode:
    def __init__(self,nodeData):
        self.nodeData = nodeData
        self.childnode_list = list()
        self.childname_list=list()
        self.count = 0

class FP_tree:
    def __init__(self,root):
        self.root = root
        self.pathTable=list()
    
    def add_tranlist(self,tranlist,parent,origin_list,path_table):
        """把tranlist加入tree中"""
        nodename=tranlist[0]
        tranlist.remove(tranlist[0])
        if nodename in parent.childname_list:
            index = parent.childname_list.index(nodename)
#             print(parent.childnode_list[index].nodeData)
            parent.childnode_list[index].count +=1
            if len(tranlist)>0:
                parent.childnode_list[index] = self.add_tranlist(tranlist,parent.childnode_list[index],origin_list,path_table)
#             print('run if',parent.childnode_list[index].nodeData)
        else:
            new_node = treeNode(nodename)
            new_node.count = 1
            parent.childname_list.append(nodename)
            parent.childnode_list.append(new_node)
            """把路徑加入node path"""
            index = origin_list.index(nodename)
            if index>0:
                path = origin_list[0:index+1]
                path_table[nodename].append(path)
            if len(tranlist)>0:
                new_node = self.add_tranlist(tranlist,new_node,origin_list,path_table)
#             print('run else')
        return parent

    # 印出 tree 的資料
    def print_tree(self,tree,layer):
        print(layer,tree.nodeData,tree.count)
        if len(tree.childname_list)>0:
            layer+=1
            for item in tree.childnode_list:
                self.print_tree(item,layer)
    
    def trace_tree(self,path,goal_item,parent):
        nodename=path[0]
        path.remove(path[0])
        index = parent.childname_list.index(nodename)
        if len(path)>0:
            parent.childnode_list[index].count
            answer = self.trace_tree(path,goal_item,parent.childnode_list[index])
            return answer
        else: 
            return parent.childnode_list[index].count
    

In [2]:
from itertools import combinations
import pandas as pd
import time 
def data_transform():
    """資料變形，把同transation放在同個row """
    file = pd.read_csv("testdata2.csv",header=None)
    temp_read=[]
    tran_list=[]  # 同transation在同個row
    itemSet_dic = dict() #單一item的數量
    tranID = 1
    for i in range(file.shape[0]):
        if file.iloc[i][1] != tranID :
            tran_list.append(temp_read)
            temp_read=[]
            tranID += 1
        temp = file.iloc[i][2]
        temp_read.append(temp)
        if temp in itemSet_dic:
            itemSet_dic[temp] += 1
        else:
            itemSet_dic[temp] = 1
            
    tran_list.append(temp_read)    
    return tran_list,itemSet_dic


def delete_item(itemSet_dic,min_support):
    """刪除低於min support的single item"""
    newitem_dic=dict()
    for item in itemSet_dic.items():
        if item[1] >= min_support:
            newitem_dic[item[0]] = item[1]
    return newitem_dic
    

def sortedTranlist(single_tranlist,itemSet_dic):
    """回傳按照數量排序的single tranlist"""
    temp=list()  # 單一tranlist中每個item的數量
    returnlist=list()
    for item in single_tranlist: # 查詢數量
        if item in itemSet_dic:
            temp.append((item,itemSet_dic[item]))
    temp_sorted = sorted(temp,key=lambda x:x[1],reverse=True) # 依照出現次數對tranlist中item進行排序

    for item in temp_sorted: # 按數量排序 只回傳照順序的key 不用回傳出現次數
        returnlist.append(item[0])  
    return returnlist


def produceRule(subSet,originSet,minConfidence):
    for i in range(len(subSet)):
        cutList = list(subSet)
        del cutList[i]
        if len(cutList)<1:
            return            
        confidence = freqSet_dic[originSet]/freqSet_dic[frozenset(cutList)]
        if confidence>=minConfidence :
            answerList=[]
            answerList.append(frozenset(cutList))
            answerList.append(frozenset(originSet)-frozenset(cutList))
            rules_dic[tuple(answerList)] = confidence
            produceRule(frozenset(cutList),originSet,minConfidence)

def combine(temp_dic,single_item,path_dic,min_support):
    return_dic=dict()
    for n in range(len(temp_dic)):
        temp_list2 = list() # 所有大小為n的組合
        for c in combinations(temp_dic, n+1):
            temp_list2.append(list(c))
        for item in temp_list2:
            calsupport = 0
            temp_dic2 = dict()
            for i in range(len(item)):
                temp = item[i]
                temp_dic2[temp] = temp_dic[temp]
            for answer in path_dic.items():
                if set(item).issubset(answer[0]):
                    calsupport += answer[1]
            if calsupport >= min_support:
                item.append(single_item)
                return_dic[frozenset(item)] = calsupport

    return return_dic

In [3]:
tstart = time.time()
"""對transation list中的item依照出現次數做排序"""
tranList,itemSet_dic = data_transform()  # 資料格式轉換
max_item = max(itemSet_dic.keys())
itemSet_dic = delete_item(itemSet_dic,100) # 刪除低於min_support的item
print(itemSet_dic)

""" 針對每筆transation進行數量排序"""
sortedlist=list() # 已經排序好的tranList
for item in tranList:
    receive = sortedTranlist(item,itemSet_dic)
    if receive!=[]:
        sortedlist.append(receive)

""" 對single item做排序"""        
itemSet_sorted=list()  # 對single item做排序
temp = sorted(itemSet_dic.items(), key=lambda x:x[1],reverse=True)
print(itemSet_dic)
for item in temp:
    itemSet_sorted.append(item[0])
print(itemSet_sorted)

""" 產生path table"""
path_table=list() # 放所有路徑
for i in range(max_item+1):
    path_table.append(list())


{3: 1135, 11: 1238, 12: 509, 14: 967, 15: 231, 17: 1125, 18: 412, 19: 461, 21: 765, 23: 752, 26: 622, 9: 966, 25: 578, 29: 954, 4: 390, 6: 450, 7: 630, 27: 199, 28: 1175, 5: 618, 8: 1180, 16: 186, 20: 226, 0: 598, 10: 506, 22: 358, 24: 301, 13: 484}
{3: 1135, 11: 1238, 12: 509, 14: 967, 15: 231, 17: 1125, 18: 412, 19: 461, 21: 765, 23: 752, 26: 622, 9: 966, 25: 578, 29: 954, 4: 390, 6: 450, 7: 630, 27: 199, 28: 1175, 5: 618, 8: 1180, 16: 186, 20: 226, 0: 598, 10: 506, 22: 358, 24: 301, 13: 484}
[11, 8, 28, 3, 17, 14, 9, 29, 21, 23, 7, 26, 5, 0, 25, 12, 10, 13, 19, 6, 18, 4, 22, 24, 15, 20, 27, 16]


In [4]:
'''把transation list放進去樹裡面'''
# 創造一個root node 跟叫做fp_growth的樹
root = treeNode('root') 
fp_growth = FP_tree(root)

# transation list放進去樹裡面
layer =0
print(sortedlist)
for item in sortedlist:
    fp_growth.add_tranlist(item.copy(),root,item.copy(),path_table)
fp_growth.print_tree(root,layer)


[[11, 3, 17, 14, 21, 23, 26, 12, 19, 18, 15], [3, 14, 9, 21], [11, 3, 17, 14, 9, 29, 25, 12], [11, 28, 3, 17, 14, 9, 29, 23, 7, 25, 6, 4, 15, 27], [11, 28, 3, 21, 5], [11, 8, 7, 26, 5, 25, 12, 16], [11, 3, 17, 9, 29, 26, 12, 20], [11, 3, 12, 27], [28, 3, 17, 29, 23, 5, 0], [11, 8, 3, 17, 29, 21, 7, 5, 0, 12, 10, 6, 4, 22], [11, 8, 3, 17, 14, 9, 23, 7, 26, 25, 22], [28, 17, 21, 7, 26, 5, 25, 19, 4], [11, 8, 28, 17, 23, 7, 5, 0, 25, 19, 4, 24, 20], [11, 8, 28, 3, 17, 14, 9, 29, 21, 23, 26, 0, 12, 6, 22], [11, 8, 3, 29, 0, 24], [8, 28, 29, 23, 0, 18, 22], [11, 28, 17, 5, 0, 25, 13], [8, 28, 9, 23, 26, 0, 13, 20], [11, 14, 9, 7, 26, 25, 13, 20], [11, 23, 26, 12, 13, 24], [11, 8, 3, 17, 14, 9, 23, 26, 0, 25, 10, 6, 18, 4, 16], [11, 28, 3, 17, 14, 9, 29, 7, 26, 5, 10, 6, 18, 16], [11, 8, 28, 3, 17, 9, 29, 5, 0, 10, 16], [11, 8, 17, 14, 21, 7, 0, 25], [11, 8, 28, 3, 17, 9, 26, 0, 4], [8, 28, 17, 9, 29, 23, 26, 5, 25, 12], [28, 17, 14, 7], [8, 28, 17, 9, 29, 21, 5, 25, 19], [11, 3, 17, 14, 7, 

8 12 1
9 19 1
10 24 1
8 7 1
9 26 1
10 25 1
11 12 1
12 6 1
13 18 1
14 15 1
8 23 1
9 7 1
10 5 1
11 25 1
12 12 1
13 18 1
7 7 1
8 0 1
9 25 1
10 13 1
11 19 1
6 21 2
7 7 1
8 26 1
9 25 1
10 12 1
11 10 1
7 26 1
8 25 1
9 19 1
10 16 1
6 23 3
7 12 1
8 10 1
9 4 1
10 16 1
7 7 2
8 10 1
9 19 1
10 18 1
11 24 1
8 25 1
9 10 1
10 13 1
11 22 1
5 9 22
6 23 3
7 5 1
8 6 1
9 24 1
7 26 2
8 25 1
9 12 1
10 19 1
11 6 1
12 4 1
8 5 1
9 25 1
10 18 1
6 21 2
7 26 2
8 19 2
9 18 2
6 7 1
7 12 1
8 10 1
9 6 1
10 15 1
6 29 15
7 23 4
8 0 2
9 25 2
10 12 1
11 22 1
12 24 1
13 20 1
8 25 1
9 22 1
10 20 1
8 7 1
9 25 1
10 18 1
11 24 1
7 21 8
8 19 1
9 15 1
8 7 1
9 25 1
10 18 1
8 23 3
9 26 1
10 18 1
11 27 1
9 12 1
10 19 1
11 18 1
12 20 1
9 7 1
10 26 1
11 0 1
12 18 1
13 4 1
8 25 1
9 13 1
10 19 1
11 18 1
12 27 1
8 5 1
9 0 1
10 4 1
11 22 1
12 16 1
8 26 1
9 25 1
10 16 1
7 6 1
7 7 2
8 0 1
9 25 1
10 12 1
11 13 1
12 6 1
8 5 1
9 0 1
10 6 1
6 26 1
7 25 1
8 18 1
5 29 8
6 23 2
7 5 1
8 12 1
9 10 1
10 18 1
7 10 1
8 24 1
6 26 1
7 25 1
8 13 1
6 21 

7 5 1
8 12 1
9 22 1
10 15 1
6 21 3
7 23 2
8 13 1
9 4 1
8 12 1
9 4 1
10 22 1
11 20 1
7 7 1
8 0 1
9 25 1
10 12 1
11 20 1
5 14 59
6 29 13
7 23 4
8 26 1
9 5 1
10 10 1
11 13 1
12 15 1
8 5 2
9 0 1
10 12 1
11 13 1
12 6 1
9 12 1
10 10 1
11 18 1
12 24 1
13 20 1
14 27 1
8 12 1
9 27 1
7 7 3
8 22 1
8 26 1
9 0 1
10 12 1
11 6 1
12 18 1
8 18 1
9 24 1
10 27 1
7 21 3
8 0 1
9 12 1
10 10 1
11 19 1
12 18 1
13 24 1
14 20 1
15 27 1
8 7 2
9 26 1
10 5 1
11 0 1
12 13 1
13 6 1
14 20 1
9 5 1
10 0 1
11 25 1
12 10 1
13 6 1
14 22 1
15 24 1
16 16 1
7 0 1
8 25 1
9 12 1
10 4 1
11 27 1
7 5 2
8 4 1
8 12 1
9 19 1
10 6 1
11 20 1
6 9 33
7 29 22
8 7 4
9 25 1
10 10 1
11 6 1
12 27 1
13 16 1
9 12 1
10 19 1
9 0 1
10 6 1
11 18 1
12 4 1
13 22 1
9 26 1
10 12 1
11 10 1
12 19 1
13 18 1
14 20 1
8 21 8
9 7 3
10 26 2
11 5 1
12 0 1
13 25 1
14 6 1
15 18 1
16 22 1
17 16 1
11 6 1
10 0 1
11 25 1
12 19 1
13 18 1
9 0 1
10 12 1
11 19 1
12 24 1
9 23 3
10 26 1
11 0 1
12 25 1
13 12 1
14 10 1
15 13 1
16 18 1
17 15 1
10 0 1
11 12 1
12 6 1
13 4 1
10

10 20 1
8 23 1
9 26 1
10 25 1
11 13 1
12 15 1
13 27 1
8 25 1
9 6 1
10 22 1
8 5 1
9 0 1
10 18 1
11 22 1
12 27 1
7 26 1
8 10 1
9 18 1
7 12 1
8 4 1
9 16 1
7 10 1
8 6 1
9 18 1
10 22 1
11 15 1
12 20 1
13 16 1
7 19 1
8 22 1
9 24 1
10 15 1
6 9 24
7 29 17
8 21 7
9 23 4
10 7 2
11 5 1
12 25 1
13 12 1
14 22 1
15 20 1
11 26 1
12 12 1
13 10 1
14 13 1
15 18 1
16 22 1
17 24 1
10 26 1
11 0 1
12 12 1
13 10 1
14 19 1
15 18 1
16 22 1
10 0 1
11 12 1
12 19 1
13 18 1
14 4 1
15 24 1
9 7 2
10 26 2
11 0 1
12 19 1
13 6 1
14 15 1
15 27 1
11 10 1
12 13 1
13 4 1
14 16 1
9 5 1
10 0 1
11 25 1
12 10 1
8 23 8
9 26 1
10 5 1
11 0 1
12 25 1
13 12 1
14 19 1
15 15 1
9 25 1
10 12 1
11 4 1
9 7 4
10 26 3
11 5 3
12 25 2
13 12 2
14 10 1
15 13 1
16 19 1
17 4 1
18 22 1
19 15 1
14 13 1
15 19 1
16 18 1
17 20 1
12 12 1
13 10 1
10 25 1
11 19 1
12 6 1
9 5 2
10 25 1
11 12 1
12 19 1
13 22 1
14 20 1
10 0 1
11 13 1
8 7 1
9 12 1
10 13 1
11 15 1
8 10 1
9 6 1
10 18 1
11 4 1
12 24 1
7 5 2
8 25 1
9 13 1
10 24 1
8 12 1
9 24 1
10 27 1
7 21 3
8 6

6 5 1
7 0 1
8 12 1
9 19 1
10 24 1
6 7 1
7 5 1
8 10 1
9 19 1
10 18 1
5 25 1
6 10 1
7 6 1
5 7 1
6 5 1
7 19 1
8 18 1
9 20 1
4 9 4
5 29 1
6 21 1
7 23 1
8 7 1
9 26 1
10 0 1
11 10 1
12 19 1
13 4 1
14 22 1
15 24 1
16 20 1
5 21 2
6 26 1
7 12 1
8 18 1
9 4 1
10 15 1
6 23 1
7 7 1
8 12 1
9 19 1
10 4 1
5 23 1
6 26 1
7 5 1
8 0 1
9 25 1
10 10 1
11 13 1
12 4 1
13 16 1
4 7 1
5 12 1
6 22 1
7 15 1
4 23 1
5 12 1
6 10 1
7 19 1
8 4 1
9 22 1
10 15 1
4 26 1
5 19 1
6 18 1
7 4 1
8 24 1
9 15 1
10 20 1
11 16 1
3 5 1
4 0 1
5 6 1
6 22 1
7 15 1
2 29 6
3 23 3
4 7 1
5 0 1
4 13 1
5 6 1
6 4 1
7 24 1
8 16 1
4 26 1
5 5 1
6 0 1
7 12 1
8 19 1
9 27 1
3 21 1
4 26 1
5 25 1
6 27 1
3 13 1
4 19 1
5 6 1
3 12 1
4 13 1
2 5 1
3 10 1
4 4 1
2 21 4
3 23 1
4 5 1
3 25 1
3 26 1
4 0 1
5 25 1
6 4 1
7 24 1
3 5 1
4 25 1
5 12 1
2 9 6
3 21 2
4 13 1
5 4 1
6 22 1
4 24 1
3 29 3
4 7 1
5 25 1
6 12 1
7 10 1
8 19 1
4 5 1
5 25 1
6 19 1
7 27 1
4 23 1
5 26 1
6 10 1
7 6 1
8 4 1
3 0 1
4 22 1
5 24 1
6 16 1
2 7 1
3 0 1
2 13 1
1 3 79
2 14 19
3 9 8
4 21 3
5 7 1

8 27 1
9 16 1
4 25 1
5 10 1
6 13 1
7 22 1
8 24 1
4 7 1
5 26 1
6 25 1
7 4 1
3 3 152
4 14 30
5 9 17
6 29 8
7 26 2
8 5 1
9 25 1
10 10 1
8 19 1
9 6 1
10 24 1
7 23 2
8 7 2
9 5 1
9 0 1
10 10 1
11 6 1
12 24 1
7 21 2
8 5 1
9 0 1
10 4 1
11 24 1
8 0 1
9 13 1
10 19 1
11 24 1
7 5 1
8 0 1
9 25 1
10 6 1
6 26 2
7 6 1
7 12 1
8 18 1
9 4 1
10 22 1
11 20 1
6 21 4
7 23 2
8 7 1
9 26 1
10 5 1
11 0 1
12 12 1
8 24 1
7 13 1
7 7 1
8 12 1
9 13 1
10 19 1
11 18 1
12 27 1
6 23 2
7 0 1
8 13 1
9 22 1
10 24 1
7 7 1
8 26 1
9 0 1
10 25 1
11 19 1
12 6 1
13 4 1
14 22 1
15 20 1
16 27 1
6 7 1
7 25 1
8 12 1
9 6 1
5 22 2
5 21 3
6 26 1
7 0 1
8 12 1
9 13 1
10 19 1
11 6 1
12 15 1
6 23 1
7 0 1
8 13 1
9 22 1
5 29 6
6 21 3
7 23 2
8 10 2
9 19 1
9 15 1
7 26 1
8 0 1
9 24 1
6 25 1
7 10 1
8 4 1
9 20 1
6 23 1
7 25 1
8 12 1
9 10 1
10 13 1
6 13 1
7 6 1
8 22 1
5 23 2
6 5 1
7 0 1
8 12 1
9 4 1
10 27 1
6 7 1
7 19 1
4 17 83
5 9 22
6 23 6
7 7 2
8 5 1
9 12 1
10 10 1
11 22 1
8 0 1
9 27 1
7 26 2
8 10 1
9 19 1
10 6 1
11 18 1
8 5 1
9 25 1
10 12 1
11 

4 29 5
5 26 1
6 5 1
7 10 1
5 23 2
6 7 2
7 5 1
8 0 1
7 26 1
8 5 1
9 25 1
10 10 1
11 13 1
12 18 1
13 4 1
14 20 1
5 21 2
6 23 1
7 6 1
8 15 1
6 26 1
4 21 1
5 13 1
3 5 3
4 12 2
5 13 1
6 6 1
7 4 1
5 24 1
4 4 1
5 24 1
3 7 1
4 25 1
5 13 1
6 22 1
3 29 2
4 21 1
5 4 1
4 26 1
5 25 1
6 15 1
3 21 2
4 23 1
5 26 1
6 10 1
7 13 1
8 19 1
9 18 1
10 15 1
11 16 1
4 26 1
5 5 1
6 12 1
7 13 1
8 19 1
9 4 1
2 10 1
2 15 1
3 16 1
2 25 1
3 24 1
1 14 23
2 10 1
3 18 1
2 29 5
3 26 1
4 22 1
3 21 2
4 7 1
5 26 1
6 25 1
7 13 1
8 22 1
4 5 1
3 23 1
4 0 1
5 25 1
6 19 1
7 27 1
3 12 1
4 24 1
2 9 10
3 6 1
4 27 1
3 13 1
3 21 3
4 7 1
5 0 1
6 12 1
7 13 1
4 26 1
5 5 1
4 23 1
5 19 1
6 6 1
7 18 1
8 22 1
3 29 2
4 7 1
5 26 1
6 25 1
7 19 1
8 24 1
4 21 1
5 23 1
6 7 1
7 26 1
3 23 2
4 5 1
5 0 1
6 13 1
7 19 1
8 18 1
9 4 1
4 26 1
5 12 1
3 4 1
2 5 1
3 12 1
4 10 1
5 20 1
2 21 3
3 7 1
4 5 1
5 25 1
6 10 1
7 13 1
3 5 1
4 0 1
5 19 1
3 13 1
4 19 1
5 4 1
2 7 1
3 5 1
4 0 1
5 25 1
6 10 1
7 4 1
2 0 1
3 12 1
4 13 1
5 4 1
2 25 1
3 10 1
1 17 33
2 14 14
3 

In [5]:
"""trace tree"""
del itemSet_sorted[0] # single item 中出現次數最多的不用找
itemSet_supportdic = dict() # pattern base的support
frequent_set = dict()
for single_item in itemSet_sorted:    
    temp_dic = dict() # single item出現次數 等等刪除低於min_support後進行排列組合
    deltemp_dic = dict() # 刪除低於minsupport的item
    path_dic = dict()
    paths_list = path_table[single_item] # 把該item所有path找出來
    for item in paths_list:
        item_copy = item.copy()
        answer = fp_growth.trace_tree(item,single_item,root)
        path_dic[frozenset(item_copy)] = answer
        
        item_copy = item_copy[0:len(item_copy)-1]
        for a in item_copy:
            if a in temp_dic.keys():
                temp_dic[a]+=answer
            else:
                temp_dic[a]=answer
#         print(temp_dic)
    for a in temp_dic.items() :  # 刪除低於minsupport的item
        if a[1] >= 100:
            key = a[0]
            deltemp_dic[key] = a[1]      
    frequent_set = {**frequent_set,**combine(deltemp_dic,single_item,path_dic,100)}
# print(len(frequent_set))
#     itemSet_supportdic[frozenset(item_copy)] = answer  # 產生pattern base

        
tEnd = time.time()       
    
print('Time',tEnd-tstart)
#     combine_dic = {**combine_dic,**combine(uncombine_dic.copy())}
#     print(combine_dic)

Time 305.7357885837555
